<a href="https://colab.research.google.com/github/murillorb/plotsBokeh/blob/main/plotCovidBokeh1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # we will use this later, so import it now

from bokeh.io import show, save, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, NumeralTickFormatter, HoverTool, Title
from bokeh.resources import CDN
from bokeh.embed import file_html
from scipy import stats
import pandas as pd

In [ ]:
df = pd.read_csv("caso_full.csv", sep=',', error_bad_lines=False, index_col=False)


In [ ]:
p = figure(plot_width=700, plot_height=700,title="Comparação casos/mortes proporcionais de COVID19: cidades com mais de"
                                                 " 500000 habitantes")

p.add_layout(Title(text="Casos/população", align="center"), "below")
p.add_layout(Title(text="Mortes/população", align="center"), "left")

In [ ]:
df = df[(df['date'] == '2021-05-18')]

df = df[(df['estimated_population_2019'] > 500000)]




df['cpp'] = df['last_available_confirmed']/df['estimated_population_2019']

df['dpp'] = df['last_available_deaths']/df['estimated_population_2019']

df = df.sort_values('cpp')


In [ ]:
res = stats.linregress(df['cpp'],
                       df['dpp'])


df['cdline'] = res.intercept + res.slope*df['cpp']

df = df.dropna()

print(df.head())

                    city  city_ibge_code  ...       dpp    cdline
1385758      Nova Iguaçu       3303500.0  ...  0.001727  0.001878
1385732  Duque de Caxias       3301702.0  ...  0.001423  0.001897
516227          São Luís       2111300.0  ...  0.001828  0.002007
2043447        Guarulhos       3518800.0  ...  0.002712  0.002022
1385716     Belford Roxo       3300456.0  ...  0.001145  0.002027

[5 rows x 21 columns]


In [ ]:
cds = ColumnDataSource(df)

tooltips = [
            ('Cidade', '@city'),
            ('Estado', '@state'),
            ('Casos', '@last_available_confirmed'),
            ('Mortes', '@last_available_deaths')
           ]



p.circle('cpp', 'dpp', size=15, line_color="navy", fill_color="blue", fill_alpha=0.5,
         source=cds, selection_color='red')
p.line('cpp', 'cdline', source=cds)

p.add_tools(HoverTool(tooltips=tooltips))

In [ ]:
output_notebook()

show(p) # show the results